In [10]:
from pathlib import Path
from brainlit.utils.Neuron_trace import NeuronTrace
from brainlit.algorithms.trace_analysis.fit_spline import GeometricGraph, compute_parameterization
from scipy.interpolate import splprep, BSpline, CubicHermiteSpline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
from scipy.interpolate import splev
from brainlit.map_neurons.map_neurons import DiffeomorphismAction, transform_GeometricGraph
import pandas as pd
import numpy as np
import h5py
from brainlit.map_neurons.map_neurons import CloudReg_Transform
from scipy.spatial.distance import cosine
from tqdm import tqdm
from cloudvolume import CloudVolume
from similaritymeasures import frechet_dist
import seaborn as sns
from statannotations.Annotator import Annotator
import time
from cloudvolume.exceptions import SkeletonDecodeError
import pickle
%matplotlib widget

## Toy example

In [ ]:
dict = {"x": [0, 0], "y": [-1, 1], "z": [0, 0], "sample": [1,2], "parent": [-1, 1]}
df = pd.DataFrame(data = dict)

G = GeometricGraph(df = df)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection="3d")
soma = np.array(G.nodes[G.root]["loc"])
spline_tree = G.fit_spline_tree_invariant()
for node in spline_tree.nodes:
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    u = np.arange(u[0], u[-1]+0.1, 0.1)
    tck = spline[0]
    pts = splev(u, tck)

    ax.plot(pts[0], pts[1], pts[2], linestyle='-',color='midnightblue',linewidth=0.8)
#ax.scatter(soma[0],soma[1],soma[2], c='darkorange', s=5)
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.grid(True)

plt.title("Original curve")

plt.show()

## Custom transformation

In [ ]:
#only move points
class Triangle_0(DiffeomorphismAction):
    def __init__(self):
        pass

    def evaluate(self, position: np.array) -> np.array:
        transformed_position = position.copy()
        #if x1 between 

        tophalf = np.logical_and(position[:,1] >= 0, position[:,1] <= 1)
        bottomhalf = np.logical_and(position[:,1] < 0, position[:,1] >= -1)

        newvals = position[tophalf]
        newvals[:,0] += position[tophalf][:,1]-1
        transformed_position[tophalf] = newvals

        newvals = position[bottomhalf]
        newvals[:,0] += -1*position[bottomhalf][:,1]-1
        transformed_position[bottomhalf] = newvals
        
        return transformed_position
    
    def D(self, position: np.array, deriv: np.array, order: int = 1) -> np.array:
        return deriv

#move points and derivatives
class Triangle_1(Triangle_0):
    def __init__(self):
        pass
    
    def D(self, position: np.array, deriv: np.array, order: int = 1) -> np.array:
        transformed_deriv = deriv.copy()

        transformed_deriv[np.logical_and(position[:,1] >= 0, position[:,1] <= 1)] = np.matmul(np.array([[1,1,0],[0,1,0],[0,0,1]]), deriv[np.logical_and(position[:,1] >= 0, position[:,1] <= 1)].T).T
        transformed_deriv[np.logical_and(position[:,1] >= -1, position[:,1] < 0)] = np.matmul(np.array([[1,-1,0],[0,1,0],[0,0,1]]), deriv[np.logical_and(position[:,1] >= -1, position[:,1] < 0)].T).T
        return transformed_deriv


In [ ]:
diff = Triangle_0()

x, y, z = np.meshgrid(np.arange(-1, 1.2, 0.25),
                      np.arange(-1, 1.2, 0.25),
                      [np.arange(-0.5, 0.6, 1)])


shp = x.shape

position = np.stack((x,y,z), axis=3)
position = np.reshape(position, [-1, 3])
transformed_position = diff.evaluate(position)
displacement = transformed_position - position
u = np.reshape(displacement[:,0], shp)
v = np.reshape(displacement[:,1], shp)
w = np.reshape(displacement[:,2], shp)

fig = plt.figure(figsize=(12, 12))
ax = fig.gca(projection='3d')

ax.quiver(x, y, z, u, v, w, length=0.2)

plt.title("Displacement Field")
plt.show()

## Perform transformation

In [ ]:
diff = Triangle_1()
G_tranformed = transform_GeometricGraph(G, diff)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection="3d")

#act on dense points
soma = np.array(G_tranformed.nodes[G_tranformed.root]["loc"])
spline_tree = G.spline_tree
for node in spline_tree.nodes:
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    tck = spline[0]

    u = np.arange(u[0], u[-1]+0.1, u[-1]-u[0])
    pts = splev(u, tck)
    pts = np.stack(pts, axis=1)
    trans_pts = diff.evaluate(pts)
    ax.plot(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], linestyle='-',linewidth=0.8, label="0th Order")

    u = np.arange(u[0], u[-1]+0.1, 0.1)
    pts = splev(u, tck)
    pts = np.stack(pts, axis=1)
    trans_pts = diff.evaluate(pts)
    ax.plot(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], linestyle='-',linewidth=0.8, label="Mapping Dense Points")

#act on derivatives
soma = np.array(G_tranformed.nodes[G_tranformed.root]["loc"])
spline_tree = G_tranformed.spline_tree
for node in spline_tree.nodes:
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    u = np.arange(u[0], u[-1]+0.1, 0.1)
    chspline = spline[0]
    pts = chspline(u)

    ax.plot(pts[:,0], pts[:,1], pts[:,2], linestyle='-',linewidth=0.8, label="1st Order")
ax.scatter(soma[0],soma[1],soma[2], c='darkorange', s=5)
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.grid(True)
plt.title("0th order Transformation")
plt.legend()
plt.show()

## Toy Example multiple segments

In [ ]:
dict = {"x": [0, -1, -1], "y": [-1, 0, 1], "z": [0, 0, 1], "sample": [1,2, 3], "parent": [-1, 1, 2]}
df = pd.DataFrame(data = dict)

G = GeometricGraph(df = df)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection="3d")

ax.scatter(df['x'].to_numpy(), df['y'].to_numpy(), df['z'].to_numpy(), linestyle='-',linewidth=0.8, label="original points")

soma = np.array(G.nodes[G.root]["loc"])
spline_tree = G.fit_spline_tree_invariant()
for node in spline_tree.nodes:
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    tck = spline[0]

    pts = splev(u, tck)
    ax.plot(pts[0], pts[1], pts[2], linestyle='-',linewidth=0.8, label="line segment")

    u = np.arange(u[0], u[-1]+0.01, 0.01)
    pts = splev(u, tck)

    ax.plot(pts[0], pts[1], pts[2], linestyle='-',linewidth=0.8, label="spline")
#ax.scatter(soma[0],soma[1],soma[2], c='darkorange', s=5)
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.grid(True)
ax.legend()

plt.title("Original curve")

plt.show()

## Gaussian warp

In [ ]:
from scipy.stats import norm

#only move points
class Gaussian_0(DiffeomorphismAction):
    def __init__(self):
        pass

    def evaluate(self, position: np.array) -> np.array:
        transformed_position = position.copy()
        transformed_position[:,0] += norm.pdf(transformed_position[:,1], scale = 0.5)
        
        return transformed_position
    
    def D(self, position: np.array, deriv: np.array, order: int = 1) -> np.array:
        return deriv

#move points and derivatives
class Gaussian_1(Gaussian_0):
    def __init__(self):
        pass
    
    def D(self, position: np.array, deriv: np.array, order: int = 1) -> np.array:
        transformed_deriv = deriv.copy()
        for i, (pos, d) in enumerate(zip(position, deriv)):
            J = np.eye(3)
            J[0,1] = norm.pdf(pos[1], scale = 0.5)*(-2*pos[1])
            transformed_deriv[i,:] = np.matmul(J, d).T

        return transformed_deriv


In [ ]:
diff = Gaussian_0()

x, y, z = np.meshgrid(np.arange(-1, 1.2, 0.25),
                      np.arange(-1, 1.2, 0.25),
                      [np.arange(-0.5, 0.6, 1)])


shp = x.shape

position = np.stack((x,y,z), axis=3)
position = np.reshape(position, [-1, 3])
transformed_position = diff.evaluate(position)
displacement = transformed_position - position
u = np.reshape(displacement[:,0], shp)
v = np.reshape(displacement[:,1], shp)
w = np.reshape(displacement[:,2], shp)

fig = plt.figure(figsize=(12, 12))
ax = fig.gca(projection='3d')

ax.quiver(x, y, z, u, v, w, length=0.2)

plt.title("Displacement Field")
plt.show()

In [ ]:
diff = Gaussian_1()
G_tranformed = transform_GeometricGraph(G, diff)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection="3d")

#act on dense points
soma = np.array(G_tranformed.nodes[G_tranformed.root]["loc"])
spline_tree = G.spline_tree
for node in spline_tree.nodes:
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    tck = spline[0]

    #trace points only
    pts = splev(u, tck)
    pts = np.stack(pts, axis=1)
    trans_pts = diff.evaluate(pts)
    ax.plot(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], linestyle='-',linewidth=0.8, label="0th Order")

    #dense spline points
    u = np.arange(u[0], u[-1]+0.01, 0.01)
    pts = splev(u, tck)
    pts = np.stack(pts, axis=1)
    trans_pts = diff.evaluate(pts)
    ax.plot(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], linestyle='-',linewidth=0.8, label="Mapping Dense Points")

#act on derivatives
soma = np.array(G_tranformed.nodes[G_tranformed.root]["loc"])
spline_tree = G_tranformed.spline_tree
for node in spline_tree.nodes:
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    u = np.arange(u[0], u[-1]+0.01, 0.01)
    chspline = spline[0]
    pts = chspline(u)

    ax.plot(pts[:,0], pts[:,1], pts[:,2], linestyle='-',linewidth=0.8, label="1st Order")
ax.scatter(soma[0],soma[1],soma[2], c='darkorange', s=5)
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.grid(True)
plt.legend()
plt.show()

## CloudReg + Toy Neuron

In [ ]:
ct = CloudReg_Transform("/Users/thomasathey/Documents/mimlab/mouselight/axon_mapping/example_cloudreg/8604_Ch_561_registration/downloop_1_v.mat",
"/Users/thomasathey/Documents/mimlab/mouselight/axon_mapping/example_cloudreg/8604_Ch_561_registration/downloop_1_A.mat")

og_coords = ct.og_coords

xmin = np.amin(og_coords[0])
xmax = np.amax(og_coords[0])
ymin = np.amin(og_coords[1])
ymax = np.amax(og_coords[1])
zmin = np.amin(og_coords[2])
zmax = np.amax(og_coords[2])

print(f"{xmin}:{xmax},{ymin}:{ymax},{zmin}:{zmax}")

xs = np.arange(xmin, xmax, (xmax-xmin)/4)
ys = np.arange(ymin, ymax, (ymax-ymin)/4)
zs = np.arange(zmin, zmax, (zmax-zmin)/4)

og_coords = np.meshgrid(xs,ys,zs, indexing='ij')
og_coords = np.array([og_coords[0].flatten(),og_coords[1].flatten(),og_coords[2].flatten()]).T


fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')

ax.scatter3D(og_coords[:,0],og_coords[:,1],og_coords[:,2], label="Grid (Target Space)")

ax = fig.gca(projection='3d')
ax.quiver(og_coords[:,0],og_coords[:,1],og_coords[:,2], np.ones((og_coords.shape[0])), np.ones((og_coords.shape[0])), np.ones((og_coords.shape[0])), length=500, label="Tangent vectors (Target Space)")

new_coords = ct.evaluate(og_coords)
print(f"average squared deformation {np.sum((og_coords-new_coords)**2)/og_coords.shape[0]}")
new_derivs = ct.D(og_coords, np.ones(og_coords.shape))
sns = []
for new_deriv in new_derivs:
    sn = cosine(new_deriv, [1,1,1])
    sns.append(sn)
print(f"average 1-cosine between derivatives {np.mean(sns)}")

ax.scatter3D(new_coords[:,0],new_coords[:,1],new_coords[:,2], label="Grid (Template Space)")
ax.quiver(new_coords[:,0],new_coords[:,1],new_coords[:,2], new_derivs[:,0],new_derivs[:,1],new_derivs[:,2], length=500, color='orange', label="Tangent vectors (Template Space)")
ax.legend()

In [ ]:
dict = {"x": [0, -2000, -4000], "y": [0, -2000, 2000], "z": [-4000, -2000, 0], "sample": [1,2, 3], "parent": [-1, 1, 2]}
df = pd.DataFrame(data = dict)

G = GeometricGraph(df = df)
spline_tree = G.fit_spline_tree_invariant()

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 2, 1, projection="3d")

ax.scatter(df['x'].to_numpy(), df['y'].to_numpy(), df['z'].to_numpy(), linestyle='-',linewidth=0.8, color='orange')
G_tranformed = transform_GeometricGraph(G, ct)
soma = np.array(G_tranformed.nodes[G_tranformed.root]["loc"])
spline_tree = G.spline_tree
for node in tqdm(spline_tree.nodes, desc="Target space"):
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    tck = spline[0]

    # trace points only
    pts = splev(u, tck)
    ax.plot(pts[0], pts[1], pts[2], linestyle='-',linewidth=0.8, label="Toy neuron (Target Space)")
    derivs = splev(u, tck, der = 1)
    ax.quiver(pts[0], pts[1], pts[2], derivs[0], derivs[1], derivs[2],length=500, color='orange')

    # spline
    u2 = np.arange(u[0], u[-1]+0.01, 0.01)
    pts2 = splev(u2, tck)
    #ax.plot(pts2[0], pts2[1], pts2[2], linestyle='-',linewidth=0.8, label="Spline (Target Space)")
ax.set_title("Target Space")
ax.legend()

ax = fig.add_subplot(1, 2, 2, projection="3d")
for node in tqdm(spline_tree.nodes, desc="Template space"):
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    tck = spline[0]
    pts = splev(u, tck)
    pts = np.stack(pts, axis=1)

    #dense line points
    tck_line, u_line = splprep(pts.T, k=1, s=0)
    u_line = np.arange(u_line[0], u_line[-1]+0.01, 0.01)
    pts_line = splev(u_line, tck_line)
    pts_line = np.stack(pts_line, axis=1)
    trans_pts = ct.evaluate(pts_line)
    ax.plot(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], linestyle='-',linewidth=0.8, color='blue', label="Dense Points Lines")

    #dense spline points
    u_spline = np.arange(u[0], u[-1]+0.01, 0.01)
    pts_spline = splev(u_spline, tck)
    pts_spline = np.stack(pts_spline, axis=1)
    trans_pts = ct.evaluate(pts_spline)
    ax.plot(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], linestyle='-',linewidth=0.8, color="orange", label="Dense Points Spline")

    #Transformed points
    trans_pts = ct.evaluate(pts)
    ax.plot(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], linestyle='-',linewidth=0.8, label="0th Order Mapping", color='green')
    ax.scatter(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], color="orange")
    derivs = splev(u, tck, der=1)
    derivs = np.stack(derivs, axis=1)
    trans_derivs = ct.D(pts, derivs)
    ax.quiver(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], trans_derivs[:,0], trans_derivs[:,1], trans_derivs[:,2],length=500, color="orange")


#act on derivatives
soma = np.array(G_tranformed.nodes[G_tranformed.root]["loc"])
spline_tree = G_tranformed.spline_tree
for node in spline_tree.nodes:
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    u = np.arange(u[0], u[-1]+0.01, 0.01)
    chspline = spline[0]
    pts = chspline(u)

    ax.plot(pts[:,0], pts[:,1], pts[:,2], linestyle='-',linewidth=0.8, label="1st Order Mapping", color='red')
ax.set_title("Template Space")
ax.scatter(soma[0],soma[1],soma[2], c='darkorange', s=5)
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.grid(True)

ax.legend()
plt.show()

## Example transforming ng coordinates (matt sample) to atlas cords

In [ ]:
vol = CloudVolume("precomputed://https://dlab-colm.neurodata.io/2022_02_02/8604/Ch_647")
vol_atlas = CloudVolume("precomputed://https://open-neurodata.s3.amazonaws.com/ara_2016/sagittal_10um/annotation_10um_2017")
pt = np.array([[5344, 2244, 1500]])

res = np.array(vol.resolution)/1000
shp = np.array(vol.shape)[:3]
og = np.multiply(shp-1,res)/2
pt = pt*res - og
print(f"New point: {pt} for og: {og}")

new_pt = ct.apply_affine(pt)

res_atlas = np.array(vol_atlas.resolution)/1000
shp_atlas = np.array(vol_atlas.shape)[:3]
og_atlas = np.multiply(shp_atlas-1,res_atlas)/2

(new_pt+og_atlas)/res_atlas

## CloudReg + Real Neuron

In [ ]:
ct = CloudReg_Transform("/Users/thomasathey/Documents/mimlab/mouselight/axon_mapping/janelia_cloudreg/precomputed_ch1_otsu_iso_registration/downloop_1_v.mat",
"/Users/thomasathey/Documents/mimlab/mouselight/axon_mapping/janelia_cloudreg/precomputed_ch1_otsu_iso_registration/downloop_1_A.mat")

og_coords = ct.og_coords

xmin = np.amin(og_coords[0])
xmax = np.amax(og_coords[0])
ymin = np.amin(og_coords[1])
ymax = np.amax(og_coords[1])
zmin = np.amin(og_coords[2])
zmax = np.amax(og_coords[2])

xs = np.arange(xmin, xmax, (xmax-xmin)/4)
ys = np.arange(ymin, ymax, (ymax-ymin)/4)
zs = np.arange(zmin, zmax, (zmax-zmin)/4)

og_coords = np.meshgrid(xs,ys,zs, indexing='ij')
og_coords = np.array([og_coords[0].flatten(),og_coords[1].flatten(),og_coords[2].flatten()]).T

fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')

ax.scatter3D(og_coords[:,0],og_coords[:,1],og_coords[:,2], label="Grid (Target Space)")

ax = fig.gca(projection='3d')
ax.quiver(og_coords[:,0],og_coords[:,1],og_coords[:,2], np.ones((og_coords.shape[0])), np.ones((og_coords.shape[0])), np.ones((og_coords.shape[0])), length=500, label="Tangent vectors (Target Space)")

new_coords = ct.evaluate(og_coords)
print(f"average squared deformation {np.sum((og_coords-new_coords)**2)/og_coords.shape[0]}")
new_derivs = ct.D(og_coords, np.ones(og_coords.shape))
sns = []
for new_deriv in new_derivs:
    sn = cosine(new_deriv, [1,1,1])
    sns.append(sn)
print(f"average 1-cosine between derivatives {np.mean(sns)}")

ax.scatter3D(new_coords[:,0],new_coords[:,1],new_coords[:,2], label="Grid (Template Space)")
ax.quiver(new_coords[:,0],new_coords[:,1],new_coords[:,2], new_derivs[:,0],new_derivs[:,1],new_derivs[:,2], length=500, color='orange', label="Tangent vectors (Template Space)")
ax.legend()

In [ ]:
vol_im = CloudVolume("precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/axon_mapping/janelia_cloudreg/ch1_otsu_iso")
shp = np.array(vol_im.shape)
res_im = np.array(vol_im.resolution)/1000
origin_im = (shp[:3] - 1) * res_im / 2

vol = CloudVolume("precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/axon_mapping/janelia_cloudreg/axons")
shp = np.array(vol.shape)
res_atlas = np.array(vol.resolution)/1000
origin_atlas = (shp[:3] - 1) * res_atlas / 2

# get coords in proper cooordinates
skel = vol.skeleton.get(11)
coords = skel.vertices/1000 - origin_im

# apply affine transform
coords = ct.apply_affine(coords)

G = GeometricGraph()
for id, coord in enumerate(coords):
    G.add_node(id, loc=coord)
for edge in skel.edges:
    G.add_edge(edge[0], edge[1])



spline_tree = G.fit_spline_tree_invariant()

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 2, 1, projection="3d")

#ax.scatter(df['x'].to_numpy(), df['y'].to_numpy(), df['z'].to_numpy(), linestyle='-',linewidth=0.8)
G_tranformed = transform_GeometricGraph(G, ct)
soma = np.array(G.nodes[G.root]["loc"])
spline_tree = G.spline_tree

for i, node in enumerate(tqdm(spline_tree.nodes, desc="Target space")):
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    tck = spline[0]

    # trace points only
    pts = splev(u, tck)
    if i == 0:
        label = "Toy neuron (Target Space)"
    else:
        label = None
    #ax.plot(pts[0], pts[1], pts[2], linestyle='-',linewidth=0.8, label=label, color="blue")
    derivs = splev(u, tck, der = 1)
    #ax.quiver(pts[0], pts[1], pts[2], derivs[0], derivs[1], derivs[2],length=500, color='orange')

    # spline
    u2 = np.arange(u[0], u[-1]+0.01, 0.01)
    pts2 = splev(u2, tck)
    if i == 0:
        label = "Spline"
    else:
        label = None
    ax.plot(pts2[0], pts2[1], pts2[2], linestyle='-',linewidth=0.8, label=label, color="orange")


ax.scatter(soma[0],soma[1],soma[2], c='blue', s=5)
ax.set_title("Target Space")
ax.legend()
ax = fig.add_subplot(1, 2, 2, projection="3d")

for i, node in enumerate(tqdm(spline_tree.nodes, desc="0th order mappings")):
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    tck = spline[0]
    pts = splev(u, tck)
    pts = np.stack(pts, axis=1)


    #dense line points
    tck_line, u_line = splprep(pts.T, k=1, s=0)
    u_line = np.arange(u_line[0], u_line[-1]+0.01, 0.01)
    pts_line = splev(u_line, tck_line)
    pts_line = np.stack(pts_line, axis=1)
    trans_pts = ct.evaluate(pts_line)
    if i == 0:
        label = "Dense Points Lines"
    else:
        label = None
    #ax.plot(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], linestyle='-',linewidth=0.8, color='blue', label=label)

    #dense spline points
    u_spline = np.arange(u[0], u[-1]+0.01, 0.01)
    pts_spline = splev(u_spline, tck)
    pts_spline = np.stack(pts_spline, axis=1)
    trans_pts = ct.evaluate(pts_spline)
    if i == 0:
        label = "Dense Points Spline"
    else:
        label = None
    ax.plot(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], linestyle='-',linewidth=0.8, color="orange", label=label)

    #Transformed points
    trans_pts = ct.evaluate(pts)
    if i == 0:
        label = "0th Order Mapping"
    else:
        label = None
    ax.plot(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], linestyle='-',linewidth=0.8, label=label, color='green')
    #ax.scatter(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], color="orange")
    derivs = splev(u, tck, der=1)
    derivs = np.stack(derivs, axis=1)
    trans_derivs = ct.D(pts, derivs)
    #ax.quiver(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], trans_derivs[:,0], trans_derivs[:,1], trans_derivs[:,2],length=500, color="orange")

#act on derivatives
soma = np.array(G_tranformed.nodes[G_tranformed.root]["loc"])
spline_tree = G_tranformed.spline_tree
for i, node in enumerate(tqdm(spline_tree.nodes, desc="1st order mappings")):
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    u = np.arange(u[0], u[-1]+0.01, 0.01)
    chspline = spline[0]
    pts = chspline(u)

    if i == 0:
        label = "1st Order Mapping"
    else:
        label = None
    ax.plot(pts[:,0], pts[:,1], pts[:,2], linestyle='-',linewidth=0.8, label=label, color='red')

ax.scatter(soma[0],soma[1],soma[2], c='blue', s=5)
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.grid(True)
ax.set_title("Template Space")
ax.legend()

plt.show()


print((soma + origin_atlas)/res_atlas)

## Compute error

In [3]:
vol = CloudVolume("precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/axon_mapping/janelia_cloudreg/axons")
shp = np.array(vol.shape)
res_im = np.array(vol.resolution)/1000
origin_im = (shp[:3] - 1) * res_im / 2

valid_ids = []
for i in range(100):
    try:
        vol.skeleton.get(i)
    except SkeletonDecodeError:
        continue
    valid_ids.append(i)

print(f"Valid IDs: {valid_ids}")

Valid IDs: [1, 4, 11, 12, 13, 14, 20, 35, 40, 45, 48, 49, 51, 53, 59, 60, 65]


In [11]:
spacing = 5
n_branches = 20
plot_branches = False

ct = CloudReg_Transform("/Users/thomasathey/Documents/mimlab/mouselight/axon_mapping/janelia_cloudreg/precomputed_ch1_otsu_iso_registration/downloop_1_v.mat",
"/Users/thomasathey/Documents/mimlab/mouselight/axon_mapping/janelia_cloudreg/precomputed_ch1_otsu_iso_registration/downloop_1_A.mat")


truths = []
errors = []
methods = []

len1s = []
len2s = []
times = []

for id in tqdm(valid_ids, desc="Processing neurons..."):
    if id > 11:
        break
    # get coords in proper cooordinates
    skel = vol.skeleton.get(id)
    coords = skel.vertices/1000 - origin_im

    # apply affine transform
    coords = ct.apply_affine(coords)

    G = GeometricGraph()
    for node_id, coord in enumerate(coords):
        G.add_node(node_id, loc=coord)
    for edge in skel.edges:
        G.add_edge(edge[0], edge[1])

    spline_tree = G.fit_spline_tree_invariant()
    G_tranformed = transform_GeometricGraph(G, ct)
    spline_tree_transformed = G_tranformed.spline_tree


    for i, node in enumerate(tqdm(spline_tree.nodes, desc="Processing branches...", leave=False)):
        spline = spline_tree.nodes[node]["spline"]
        u = spline[1]
        tck = spline[0]
        pts = splev(u, tck)
        pts = np.stack(pts, axis=1)

        #print("dense line points...")
        tck_line, _ = splprep(pts.T, k=1, s=0, u=u)
        u_dense = np.arange(u[0], u[-1], spacing)
        u_dense = np.append(u_dense, u[-1])
        pts_line = splev(u_dense, tck_line)
        pts_line = np.stack(pts_line, axis=1)
        dense_line_pts = ct.evaluate(pts_line)

        #print("dense spline points...")
        pts_spline = splev(u_dense, tck)
        pts_spline = np.stack(pts_spline, axis=1)
        dense_spline_pts = ct.evaluate(pts_spline)

        # Mappings
        spline = spline_tree_transformed.nodes[node]["spline"]
        chspline = spline[0]
        u = spline[1]
        u_first_order = np.arange(u[0], u[-1], spacing)
        u_first_order = np.append(u_first_order, u[-1])
        trans_pts = chspline(u)

        #print("0th Order Mapping...")
        u_line = compute_parameterization(trans_pts)
        tck_line, u_line = splprep(trans_pts.T, k=1, s=0, u=u_line)
        u_line = np.arange(u_line[0], u_line[-1]+spacing, spacing)
        u_line = np.append(u_line, u[-1])
        zero_order_pts = splev(u_line, tck_line)
        zero_order_pts = np.stack(zero_order_pts, axis=1)

        #print("1st order mapping...")
        first_order_pts = chspline(u_first_order)
        
        truths_list = {"Linear Spline Interpolation": dense_line_pts, "Cubic Spline Interpolation": dense_spline_pts}
        methods_list = {"0th Order Action": zero_order_pts, "1st Order Action": first_order_pts}

        plot_error = False
        for truthkey in tqdm(truths_list.keys(), desc="Different ground truths...", leave=False, disable=True):
            truth = truths_list[truthkey]
            for methodkey in tqdm(methods_list.keys(), desc="Different ground truths...", leave=False, disable=True):
                method = methods_list[methodkey]
                #print(f"{truthkey}-{methodkey}: {truth.shape} {method.shape}")
                methods.append(methodkey)
                truths.append(truthkey)

                if len(truth) > 1000 or len(method) > 1000:
                    pass #raise ValueError(f"truth: {len(truth)} method: {len(method)} for spacing: {spacing}")
                len1s.append(len(truth))
                len2s.append(len(method))
                st = time.time()
                f_dist = frechet_dist(truth, method)
                if f_dist > 100:
                    plot_error=True
                    print(f"{methodkey}-{truthkey}: {f_dist} (time: {elapsed_t})")
                    
                et = time.time()
                elapsed_t = et-st
                times.append(elapsed_t)
                errors.append(f_dist)
                

        
        if plot_branches or plot_error:
            fig = plt.figure(figsize=(10, 10))
            ax = fig.add_subplot(1, 1, 1, projection="3d")
            ax.plot(dense_line_pts[:,0], dense_line_pts[:,1], dense_line_pts[:,2], linestyle='-',linewidth=0.8, color='blue', label="Dense Points Lines")
            ax.plot(dense_spline_pts[:,0], dense_spline_pts[:,1], dense_spline_pts[:,2], linestyle='-',linewidth=0.8, color="orange", label="Dense Points Spline")
            ax.plot(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], linestyle='-',linewidth=0.8, label="0th Order Mapping", color='green')
            ax.plot(first_order_pts[:,0], first_order_pts[:,1], first_order_pts[:,2], linestyle='-',linewidth=0.8, label="1st Order Mapping", color='red')

            ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
            ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
            ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
            ax.grid(True)
            ax.set_title("Template Space")
            ax.legend()

            plt.show()

    fname = f"/Users/thomasathey/Documents/mimlab/mouselight/axon_mapping/figures/errors-through-{id}.pickle"
    data = {"Method": methods, "Frechet Distance": errors, "Ground Truth": truths}
    with open(fname, 'wb') as handle:
        pickle.dump(data, handle)


data = {"Method": methods, "Frechet Distance": errors, "Ground Truth": truths}
df = pd.DataFrame(data=data)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)

fig_args = {
    "y": "Frechet Distance",
    "x": "Ground Truth",
    "hue": "Method",
    "data": df,
}
pairs = [(("Linear Spline Interpolation", "0th Order Action"), ("Linear Spline Interpolation", "1st Order Action")), (("Cubic Spline Interpolation", "0th Order Action"), ("Cubic Spline Interpolation", "1st Order Action"))]

annotator = Annotator(ax, pairs, **fig_args)
annotator.configure(comparisons_correction="Bonferroni")

sns.boxplot(ax=ax, **fig_args, boxprops=dict(alpha=.3))
splot = sns.stripplot(ax=ax, **fig_args)
annotator.configure(test="Mann-Whitney", text_format="star", loc="outside")
annotator.new_plot(splot, plot="stripplot", **fig_args)
annotator.apply_and_annotate()
ax.set_title(f"Spacing: {spacing} um")
plt.show()

# #plot timing data
# products = [i*j for i,j in zip(len1s, len2s)]
# fig = plt.figure(figsize=(10, 10))
# ax = fig.add_subplot(1, 1, 1)
# ax.scatter(products, times)
# plt.show()

Processing neurons...:  12%|█▏        | 2/17 [15:22<1:55:17, 461.16s/it]


KeyboardInterrupt: 

In [ ]:
vol_atlas = CloudVolume("precomputed://https://open-neurodata.s3.amazonaws.com/ara_2016/sagittal_10um/annotation_10um_2017")
shp_atlas = np.array(vol_atlas.shape)
res_atlas = np.array(vol_atlas.resolution)/1000
origin_atlas = (shp[:3] - 1) * res_atlas / 2

## Linear Transformation

In [ ]:
class LinearTransformation(DiffeomorphismAction):
    def __init__(self, A, b):
        self.A = A
        self.b =  b

    def evaluate(self, position: np.array) -> np.array:
        transformed_position = np.matmul(self.A, position.T).T
        transformed_position = np.add(transformed_position, self.b)
        return transformed_position
    
    def D(self, position: np.array, deriv: np.array, order: int = 1) -> np.array:
        transformed_deriv = np.matmul(self.A, deriv.T).T
        return transformed_deriv


## Real Neuron

In [ ]:
brainlit_path=Path.cwd().parent.parent.parent
swc=Path.joinpath(brainlit_path,'data','data_octree','consensus-swcs','2018-08-01_G-002_consensus.swc')
nt = NeuronTrace(path=str(swc))
df = nt.get_df()
neuron = GeometricGraph(df=df)


In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection="3d")
soma = np.array(neuron.nodes[neuron.root]["loc"])
spline_tree = neuron.fit_spline_tree_invariant()
for node in spline_tree.nodes:
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    tck = spline[0]

    u = np.arange(u[0], u[-1]+0.1, 0.1)
    pts = splev(u, tck)
    pts = np.stack(pts, axis=1)
    trans_pts = diff.evaluate(pts)
    ax.plot(trans_pts[:,0], trans_pts[:,1], trans_pts[:,2], linestyle='-',color='midnightblue',linewidth=0.8, label="Mapping Dense Points")

ax.scatter(soma[0],soma[1],soma[2], c='darkorange', s=5)
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.grid(True)


plt.show()

In [ ]:
diff = LinearTransformation(A = np.array([[2,0,0],[0,1,0],[0,0,1]]), b = [1000,0,0])
neuron_tranformed = transform_GeometricGraph(neuron, diff)

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection="3d")
soma = np.array(neuron_tranformed.nodes[neuron.root]["loc"])
spline_tree = neuron_tranformed.spline_tree
for node in spline_tree.nodes:
    spline = spline_tree.nodes[node]["spline"]
    u = spline[1]
    u = np.arange(u[0], u[-1]+0.1, 0.1)
    chspline = spline[0]
    pts = chspline(u)

    ax.plot(pts[:,0], pts[:,1], pts[:,2], linestyle='-',color='midnightblue',linewidth=0.8, label="Mapping Endpoints and Derivatives")
ax.scatter(soma[0],soma[1],soma[2], c='darkorange', s=5)
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.grid(True)


plt.show()